Demo of current progress with Dataoob

In [1]:
# Imports
import sklearn.metrics as metrics
import torch

/Users/kevinjiang/opt/anaconda3/envs/data-oob/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dataoob.dataloader import data_loading, utils
device = torch.device("mps")

def cast(*args): # TODO mps
    return [
        torch.from_numpy(arg).to(dtype=torch.float32, device=device) for arg in args
    ]

Loading Data

In [4]:
# Data loading
dict_no = {}
dict_no["train"] = 1000
dict_no["valid"] = 400

print("Loading Data")
noise_idx = data_loading.load_tabular_data('adult', dict_no, 0.1)
print("Data Loaded")


Loading Data


URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [3]:
(
    x_train,
    y_train,
    x_valid,
    y_valid,
    x_test,
    y_test,
    col_names,
) = data_loading.preprocess_data("minmax", "train.csv", "valid.csv", "test.csv")


In [4]:
# Prepare data
x1, y1, x2, y2 = cast(x_train, y_train, x_valid, y_valid)
y1, y2 = utils.one_hot_encode(y1, y2)

Setting up the models and default arguments

In [5]:
# Imports
from dataoob.model import ann, logistic_regression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from dataoob.model import ClassifierSkLearnWrapper, ClassifierUnweightedSkLearnWrapper

In [6]:
models = {
    # Wrappers for sklearn modles, makes the api more cohesive
    'sklogreg': ClassifierSkLearnWrapper(LogisticRegression(), device=device),
    'logreg': logistic_regression.LogisticRegression(x_train.shape[1]),
    'ann': ann.ANN(2),
    'knn': ClassifierUnweightedSkLearnWrapper(KNeighborsClassifier(2), device=device)
}

Some metrics

In [7]:
roc =  lambda a, b: metrics.roc_auc_score(a.detach().cpu(), b.detach().cpu())
acc = lambda a, b: metrics.accuracy_score(torch.argmax(a).detach().cpu(), torch.argmax(b).detach().cpu())

Selecting your metrics and model

In [8]:
model = models['sklogreg']
metric = roc

DVRL

In [10]:
from dataoob.dataval.dvrl.dvrl import DVRL
dvrl = DVRL(
    pred_model=model,
    metric=metric,
    x_dim=x_train.shape[1],
    y_dim=2,
    hidden_dim=100,
    layer_number=5,
    comb_dim=10,
    act_fn=torch.nn.ReLU(),
    device=device
)


In [11]:
dvrl.input_data(x1, y1, x2, y2)

dvrl.train_data_values(batch_size=128, rl_epochs=2000)
e = dvrl.evaluate_data_values(x1, y1)


  0%|          | 0/2000 [00:00<?, ?it/s]

dvrl_perf=0.8254287941787941
reward_curr=-0.0630197505197505
torch.mean(sel_prob_curr)=tensor(0.6250, device='mps:0', grad_fn=<MeanBackward0>)
loss=tensor(-5.4184, device='mps:0', grad_fn=<AddBackward0>)


/Users/kevinjiang/opt/anaconda3/envs/data-oob/lib/python3.11/site-packages/torch/_tensor_str.py:115: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/miniforge3/conda-bld/pytorch-recipe_1675740423504/work/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(
  1%|          | 24/2000 [00:02<01:40, 19.65it/s]

dvrl_perf=0.7892736486486487
reward_curr=-0.09917489604989593
torch.mean(sel_prob_curr)=tensor(0.5234, device='mps:0', grad_fn=<MeanBackward0>)
loss=tensor(-8.8080, device='mps:0', grad_fn=<AddBackward0>)


  2%|▏         | 45/2000 [00:03<01:24, 23.24it/s]

...

dvrl_perf=0.8293269230769231
reward_curr=-0.05912162162162149
torch.mean(sel_prob_curr)=tensor(0.5938, device='mps:0', grad_fn=<MeanBackward0>)
loss=tensor(-4.9829, device='mps:0', grad_fn=<AddBackward0>)


 43%|████▎     | 862/2000 [00:38<01:02, 18.12it/s]

dvrl_perf=0.8483952702702702
reward_curr=-0.040053274428274444
torch.mean(sel_prob_curr)=tensor(0.6250, device='mps:0', grad_fn=<MeanBackward0>)
loss=tensor(-3.2765, device='mps:0', grad_fn=<AddBackward0>)


 43%|████▎     | 864/2000 [00:38<00:50, 22.47it/s]


KeyboardInterrupt: 

Data Shap

In [25]:
from dataoob.dataval.shap.shap import ShapEvaluator
ShapEvaluator(
    pred_model=model,
    metric=metric,
    GR_threshold=1.01
)

TypeError: ShapEvaluator.__init__() missing 1 required positional argument: 'pred_model'

KNN